In [ ]:
!pip install tmdbv3api

# Com todas colunas do Top Rated de Movies apenas por Terror

In [ ]:
from tmdbv3api import TMDb, Discover, Genre
from datetime import datetime, timedelta
import boto3
import json
from botocore.exceptions import ClientError
import logging
import os

session = boto3.Session(aws_access_key_id='xxxxxx',
                        aws_secret_access_key='xxxxxxx',
                        aws_session_token='xxxxxxxxxx'
                       )               

with open('api_key_tmbd.txt') as arquivo:
    api_key = arquivo.read()

tmdb = TMDb()
tmdb.api_key = api_key
tmdb.language = 'pt-BR'

# metodos TMDB para achar os filmes
discover = Discover()
genre = Genre()

# Obtenha a lista de gêneros de filmes
genres = genre.movie_list()

# Crie um dicionário para mapear IDs de gênero para nomes de gênero
genre_dict = {g['id']: g['name'] for g in genres}

# Parametros para pegar as datas do intervalo
params = {
    'with_genres': 27 # ID do gênero de terror
}

# count contar a quantidade de filmes achados
# escolher a quantidade de paginas da API
count = 0
movies_data = []

for page in range(1, 101):
    params['page'] = page
    movies = discover.discover_movies(params)
    count += len(movies)


    for movie in movies:
        print(movie)
        movie_data = movie.__dict__.copy()
        movie_data['genres'] = [genre_dict[g] for g in movie['genre_ids']]
        movies_data.append(movie_data)

print(f'Número de filmes: {count}')

# Converta a lista de dados dos filmes em formato JSON
# movies_json = json.dumps(movies_data)
movies_json = json.dumps(movies_data, ensure_ascii=False)

# Crie uma conexão com o S3
s3 = session.client('s3')

# Especifique o caminho completo no bucket S3
bucket_name = 'data-lake-heliton'  # Substitua pelo nome do seu bucket no S3
current_date = datetime.now()
year = current_date.strftime('%Y')
month = current_date.strftime('%m')
day = current_date.strftime('%d')
object_key = f'RAW/TMDB/JSON/{year}/{month}/{day}/filmes_popular.json'

# Envie o JSON para o bucket no S3
s3.put_object(
    Body=movies_json,
    Bucket=bucket_name,
    Key=object_key
)